In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Sales Forecasting For Gufhtugu Publishers

In [ ]:
#Importing all Necessary libraries For EDA

import numpy as np
import pandas as pd
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings("ignore")


# univariate lstm example
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

In [ ]:
#Reading Required Data

df = pd.read_csv("../input/gufhtugu-publications-dataset-challenge/GP Orders - 4.csv",encoding="utf-8", delimiter=',')
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
statistics_of_data = []
for col in df.columns:
  statistics_of_data.append((col,
                             df[col].nunique(),
                             df[col].isnull().sum()*100/df.shape[0],
                             df[col].value_counts(normalize=True, dropna=False).values[0] * 100, 
                             df[col].dtype
                             ))
stats_df = pd.DataFrame(statistics_of_data, columns=['Feature', 'Uniq_val', 'missing_val', 'val_biggest_cat', 'type'])

In [ ]:
stats_df.sort_values('Uniq_val', ascending=False)

In [ ]:
df.dropna(inplace=True)

In [ ]:
df1 = df.copy()

In [ ]:
### Renaming columns name
df1 = df1.rename(columns={'Order Number': 'Ord_num','Order Date':'Ord_Date','City (Billing)':'City',
                          'Book Name':'Book_Name','Order Status':'Ord_Status'
                         
                         })
df1.head()

In [ ]:
df1.shape

In [ ]:
#Shout out muhammadismail99 for this function

from itertools import chain

# return list from series of comma-separated strings
def chainer(s):
    return list(chain.from_iterable(s.str.split('/')))

# calculate lengths of splits
df2 = df1.copy()
lens = df2['Book_Name'].str.split('/').map(len)

# create new dataframe, repeating or chaining as appropriate
df2 = pd.DataFrame({'Ord_num': np.repeat(df2['Ord_num'], lens),
                    'Ord_Status': np.repeat(df2['Ord_Status'], lens),
                    'Book_Name': chainer(df2['Book_Name']),
                    'Ord_Date': np.repeat(df2['Ord_Date'], lens),
                    'City': np.repeat(df2['City'], lens)})

In [ ]:
df2.shape

In [ ]:
df2['City'] = df2['City'].str.lower()
df2['City'] = df2['City'].apply(lambda x: x.strip(''))
city_stats = df2['City'].value_counts(ascending=False)
city_stats.head()

In [ ]:
def ReplaceString(searchterm,replacedWith,data):
    search_terms = searchterm
    matches = []
    for memo_string in data:#df['City (Billing)']:
        for word in search_terms.split(" "):
            if word not in memo_string:
                break
            else:
                #df['City (Billing)'].replace(memo_string,'karachi',inplace=True)
                data.replace(memo_string,replacedWith,inplace=True)
                matches.append(memo_string) # triggers when the for loop doesn't break

In [ ]:
df3 = df2.copy()

In [ ]:
ReplaceString('karachi','karachi',df3['City'])

In [ ]:
ReplaceString('lahore','lahore',df3['City'])

In [ ]:
ReplaceString('islamabad','islamabad',df3['City'])

In [ ]:
ReplaceString('rawalpindi','rawalpindi',df3['City'])

In [ ]:
ReplaceString('faisalabad','faisalabad',df3['City'])

In [ ]:
ReplaceString('multan','multan',df3['City'])

In [ ]:
ReplaceString('hyderabad','hyderabad',df3['City'])

In [ ]:
ReplaceString('????? ??? ??? ???? ??????','unknown',df3['City'])
ReplaceString('???????','unknown',df3['City'])
ReplaceString('??? ???','unknown',df3['City'])
ReplaceString('????????','unknown',df3['City'])
ReplaceString('?????','unknown',df3['City'])
ReplaceString('????','unknown',df3['City'])
ReplaceString('??????','unknown',df3['City'])
ReplaceString('???????','unknown',df3['City'])

In [ ]:
df3.shape

In [ ]:
len(df3['City'].unique()) ## Everything ('2869') is validated till here

In [ ]:
df4 = df3.copy()

In [ ]:
## Cities having orders less than 12 cardinality are termed as others

city_stats_less_than_12 = city_stats[city_stats<=12]

df4['City'] =df4['City'].apply(lambda x: 'other' if x in city_stats_less_than_12 else x)
len(df4['City'].unique()) ##Everything ('195') is validated till here

In [ ]:
# Formatting Order Status Feature
df5 = df4.copy()
combine = [df5]
titlemapping = {'Canceled':0, 'Completed':1,'Returned':2}
for row in combine:
    row["Ord_Status"] = row["Ord_Status"].map(titlemapping)
    row['Ord_Status'] = row['Ord_Status'].fillna(0)
    row['Ord_Status'] = row['Ord_Status'].astype(int)

In [ ]:
df5.shape

In [ ]:
df6 = df5.copy()

In [ ]:
# #Thanks to @hussainsaddam12 & @mnavaidd for this codeblock idea
df6["Ord_Date"] = pd.DatetimeIndex(df6["Ord_Date"])
df6['date'] = df6['Ord_Date'].dt.date
df6['time'] = df6['Ord_Date'].dt.time
df6["Day_Name"] = df6["Ord_Date"].dt.day_name()
df6["Month_Name"] = df6["Ord_Date"].dt.month_name()
df6['year'] = df6["Ord_Date"].dt.year

In [ ]:
df6.head()

In [ ]:
df6.shape  ## 5 columns added ## Everything till here is validated (33091,10)

In [ ]:
len(df6['Book_Name'].unique())

In [ ]:
#lets take an average of 500 rupees (PKR) as cost of each book
df6['Avgprice'] = 500
df6.head()

In [ ]:
#Average sale per month

#Books Sold in jan
df7 = df6.copy()
df7 =df7[['Month_Name','Avgprice','Book_Name','Ord_Status','year']]
df7.head()

In [ ]:
df7['Month_Name'].unique()

In [ ]:
def AvgSale(df, month,year):
    df = df[(df['Month_Name'] == month) & (df['Ord_Status'] == 1) & (df['year'] == year)] ##Order status = success
    AvgSale22 = 500*len(df)
    return AvgSale22


In [ ]:
#Testing
jan2020_AvgSale = AvgSale(df7,'January',2020)
jan2020_AvgSale 

In [ ]:
jan2019_AvgSale = AvgSale(df7,'January',2019)
feb2019_AvgSale = AvgSale(df7,'February',2019)
mar2019_AvgSale = AvgSale(df7,'March',2019)
apr2019_AvgSale = AvgSale(df7,'April',2019)
may2019_AvgSale = AvgSale(df7,'May',2019)
june2019_AvgSale = AvgSale(df7,'June',2019)
july2019_AvgSale = AvgSale(df7,'July',2019)
aug2019_AvgSale = AvgSale(df7,'August',2019)
sept2019_AvgSale = AvgSale(df7,'September',2019)
oct2019_AvgSale = AvgSale(df7,'October',2019)
nov2019_AvgSale = AvgSale(df7,'November',2019)
dec2019_AvgSale = AvgSale(df7,'December',2019)



jan2020_AvgSale = AvgSale(df7,'January',2020)
feb2020_AvgSale = AvgSale(df7,'February',2020)
mar2020_AvgSale = AvgSale(df7,'March',2020)
apr2020_AvgSale = AvgSale(df7,'April',2020)
may2020_AvgSale = AvgSale(df7,'May',2020)
june2020_AvgSale = AvgSale(df7,'June',2020)
july2020_AvgSale = AvgSale(df7,'July',2020)
aug2020_AvgSale = AvgSale(df7,'August',2020)
sept2020_AvgSale = AvgSale(df7,'September',2020)
oct2020_AvgSale = AvgSale(df7,'October',2020)
nov2020_AvgSale = AvgSale(df7,'November',2020)
dec2020_AvgSale = AvgSale(df7,'December',2020)


jan2021_AvgSale = AvgSale(df7,'January',2021)

In [ ]:
d = {'Month': ['2019-01-01', '2019-02-01','2019-03-01','2019-04-01','2019-05-01',
               '2019-06-01','2019-07-01','2019-08-01','2019-09-01','2019-10-01',
               '2019-11-01','2019-12-01',
               
               '2020-01-01', '2020-02-01','2020-03-01','2020-04-01','2020-05-01',
               '2020-06-01','2020-07-01','2020-08-01','2020-09-01','2020-10-01',
               '2020-11-01','2020-12-01',
               
               '2021-01-01'], 'AvgSales': [jan2019_AvgSale,feb2019_AvgSale, mar2019_AvgSale,apr2019_AvgSale,may2019_AvgSale,
                                       
                                       june2019_AvgSale,july2019_AvgSale,aug2019_AvgSale,sept2019_AvgSale,oct2019_AvgSale,
                                        
                                       nov2019_AvgSale,dec2019_AvgSale,
                                        
                                        jan2020_AvgSale,feb2020_AvgSale,mar2020_AvgSale,apr2020_AvgSale,may2020_AvgSale,
                                        
                                        june2020_AvgSale,july2020_AvgSale,aug2020_AvgSale,sept2020_AvgSale,oct2020_AvgSale,
                                        nov2020_AvgSale,dec2020_AvgSale,jan2021_AvgSale
                                        
                                       ]}

In [ ]:
df8 = pd.DataFrame(data=d)

In [ ]:
df8.head(20)

In [ ]:
df8.tail(10)

In [ ]:
df8.describe()

In [ ]:
#Visualizing data

df8.plot()

### Yup, Growth is amazing

In [ ]:
# preparing independent and dependent features
def prepare_data(timeseries_data, n_features):
	X, y =[],[]
	for i in range(len(timeseries_data)):
		# find the end of this pattern
		end_ix = i + n_features
		# check if we are beyond the sequence
		if end_ix > len(timeseries_data)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = timeseries_data[i:end_ix], timeseries_data[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [ ]:
# define input sequence
timeseries_data = df8['AvgSales']
# choose a number of time steps
n_steps = 3
# split into samples
X, y = prepare_data(timeseries_data, n_steps)

In [ ]:
print(X),print(y)

In [ ]:
X.shape

In [ ]:
## reshaping

n_features = 1
X = X.reshape((X.shape[0],X.shape[1],n_features))

In [ ]:
X.shape

### Building LSTM Model

In [ ]:

# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=300, verbose=1)

In [ ]:
# demonstrate prediction for next 10 Months
x_input = np.array([1722000, 2404000, 3136000])
temp_input=list(x_input)
lst_output=[]
i=0
while(i<10):
    
    if(len(temp_input)>3):
        x_input=np.array(temp_input[1:])
        print("{} Months input {}".format(i,x_input))
        #print(x_input)
        x_input = x_input.reshape((1, n_steps, n_features))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} Month output {}".format(i,yhat))
        temp_input.append(yhat[0][0])
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.append(yhat[0][0])
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps, n_features))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.append(yhat[0][0])
        lst_output.append(yhat[0][0])
        i=i+1
    

print(lst_output)

In [ ]:
timeseries_data

In [ ]:
len(timeseries_data)

In [ ]:
lst_output

### Visualizaing The Output

In [ ]:
lst_output

In [ ]:
day_new=np.arange(1,26)
day_pred=np.arange(26,36)

In [ ]:
timeseries_data

In [ ]:
day_new

In [ ]:
day_pred

In [ ]:
plt.plot(day_new,timeseries_data)
plt.plot(day_pred,lst_output)

In [ ]:
lst_output

### Above is the sale prediction of gufhtugu publishers for next 10 months (From Feb 2021 - Nov 2021)

Thats all for Average sales prediction (Taking an average of 500 PKR per Book) for Gufhtugu Publishers, Up vote it if you liked it.